Rivarola.Ester_A5 

(This code was run with the help of a tutor on Chegg)

In [1]:
#import all required packages

import pandas as pd
import nltk.stem
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import naive_bayes
from sklearn import tree
from sklearn.metrics import roc_auc_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


_______________________________________________________________________________________________________________________
Hotel 1:


Doubletree By Hilton 

341 West 36th Street, New York City, NY 10018-640
    
Rating: 3.5
_______________________________________________________________________________________________________________________

In [2]:
# in 587, we learned how to use pandas and numpy to read csv files 

df=pd.read_csv("Rivarola.Ester-A5_Hotel1.csv",usecols=[0,1],names=["rating","review"],encoding = "ISO-8859-1")
df.head(5)

,rating,review
0,2.0,"The hotel is in a good location, and the staff..."
1,5.0,Me and my husband booked this hotel from a caf...
2,2.0,Normally I find success with Hilton properties...
3,2.0,We had a top floor (26th) room. The issue was ...
4,3.0,The hotel is very close to time square and Jav...


Storing all the rating values as target variable. We are converting the rating in string format as it is a categorical variable and would be used as a target value to the classifier.

In [3]:
import numpy as np
target=[]
for i in df.rating:
    if(i==1):
        target.append("1")
    elif(i==2):
        target.append("2")
    elif(i==3):
        target.append("3")
    elif(i==4):
        target.append("4")
    else:
        target.append("5")
y=np.array(target)

Next step involves the preprocessing of the review data. Preprocessing steps involve removing the stop words, stemming all the words, and using Tfidf to remove all the insignificant words(words of my tutor on Chegg.com)

In [4]:
english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer=super(TfidfVectorizer,self).build_analyzer()
        return lambda doc:(english_stemmer.stem(w) for w in analyzer(doc))
vectorizer=StemmedTfidfVectorizer(min_df=1,stop_words='english',decode_error='ignore')

In [5]:
X=vectorizer.fit_transform(df.review)

In [6]:
print(y.shape)
print(X.shape)

(350,)
(350, 2327)


Split the data set into train and test data. We will train th emodel using the train dataset. Train and test are 80% and 20% respectively

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=70)

We create two models, one using the Naive Bayes and other using the Decision tree. We will later compare their performance.

In [9]:
naive_clf=naive_bayes.MultinomialNB()
naive_clf=naive_clf.fit(X_train,y_train)
dec_clf=tree.DecisionTreeClassifier()
dec_clf=dec_clf.fit(X_train,y_train)

Now we will create confusion matrix to get an insight of performance of both the model.

In [10]:
from sklearn.metrics import confusion_matrix
y_naivepred=naive_clf.predict(X_test)
y_decpred=dec_clf.predict(X_test)
naivematrix=confusion_matrix(y_test, y_naivepred)
decmatrix=confusion_matrix(y_test, y_decpred)
print(naivematrix)

[[ 2  1 13  4  0]
 [ 1  0  6  3  0]
 [ 0  1 14  9  0]
 [ 0  0  4 14  0]
 [ 0  0  4  9  3]]


In [11]:
print(decmatrix)

[[12  3  1  4  0]
 [ 2  2  3  3  0]
 [ 1  7  7  8  1]
 [ 0  1  3 10  4]
 [ 0  1  5  7  3]]


Finding the values of True Positive, True Negative, False Positive and False Negative of both the models using the corresponding confusion matrices. 

In [12]:
naive_tp=[naivematrix[i][i] for i in range(5)]
dec_tp=[decmatrix[i][i] for i in range(5)]

naive_fp=[]
dec_fp=[]
i=0
while(i<5):
    naive_sums=0
    dec_sums=0
    j=0
    while(j<5):
        if(i!=j):
            naive_sums+=naivematrix[j][i]
            dec_sums+=decmatrix[j][i]
        j+=1
    naive_fp.append(naive_sums)
    dec_fp.append(dec_sums)
    i+=1

naivediag=0
decdiag=0
i=0
while(i<5):
    naivediag+=naivematrix[i][i]
    decdiag+=decmatrix[i][i]
    i+=1

naive_tn=[naivediag-naivematrix[i][i] for i in range(5)]
dec_tn=[decdiag-decmatrix[i][i] for i in range(5)]

naive_fn=[]
dec_fn=[]
i=0
while(i<5):
    naive_sums=0
    dec_sums=0
    j=0
    while(j<5):
        if(i!=j):
            naive_sums+=naivematrix[i][j]
            dec_sums+=decmatrix[i][j]
        j+=1
    naive_fn.append(naive_sums)
    dec_fn.append(dec_sums)
    i+=1

print("TP-Naive Bayes")
print(naive_tp)
print("TP-Decision Tree")
print(dec_tp)
print("FP-Naive Bayes")
print(naive_fp)
print("FP-Decision Tree")
print(dec_fp)
print("TN-Naive Bayes")
print(naive_tn)
print("TN-Decision Tree")
print(dec_tn)
print("FN-Naive Bayes")
print(naive_fn)
print("FN-Decision Tree")
print(dec_fn)

TP-Naive Bayes
[2, 0, 14, 14, 3]
TP-Decision Tree
[12, 2, 7, 10, 3]
FP-Naive Bayes
[1, 2, 27, 25, 0]
FP-Decision Tree
[3, 12, 12, 22, 5]
TN-Naive Bayes
[31, 33, 19, 19, 30]
TN-Decision Tree
[22, 32, 27, 24, 31]
FN-Naive Bayes
[18, 10, 10, 4, 13]
FN-Decision Tree
[8, 8, 17, 8, 13]


Calculate the values of precision, recall and F1-Score for both the models.

Using the formulas below:

Precision=True Positive/(True Positive + False Positive)

Recall=True Positive/(True Positive + False Negative)

F1 Score=(2*True Positive)/(2*True Positive + False Positive + False Negative)

In [16]:
naive_precision=[]
naive_recall=[]
naive_f1_score=[]

dec_precision=[]
dec_recall=[]
dec_f1_score=[]

# precision
i=0
while(i<5):
    if(naive_tp[i]==0 and naive_fp[i]==0):
        naive_precision.append(-1)
    else:
        naive_precision.append(naive_tp[i]/(naive_fp[i]+naive_tp[i]))
    if(dec_tp[i]==0 and dec_fp[i]==0):
        dec_precision.append(-1)
    else:
        dec_precision.append(dec_tp[i]/(dec_fp[i]+dec_tp[i]))
    i+=1

# recall
i=0
while(i<5):
    if(naive_tp[i]==0 and naive_fn[i]==0):
        naive_recall.append(-1)
    else:
        naive_recall.append(naive_tp[i]/(naive_fn[i]+naive_tp[i]))
    if(dec_tp[i]==0 and dec_fn[i]==0):
        dec_recall.append(-1)
    else:
        dec_recall.append(dec_tp[i]/(dec_fn[i]+dec_tp[i]))
    i+=1

# f1 score 
i=0
while(i<5):
    if(naive_tp[i]==0 and naive_fp[i]==0 and naive_fn[i]==0):
        naive_f1_score.append(-1)
    else:
        naive_f1_score.append((2*naive_tp[i])/((2*naive_tp[i])+naive_fp[i]+naive_fn[i]))
    if(dec_tp[i]==0 and dec_fp[i]==0 and dec_fn[i]==0):
        dec_f1_score.append(-1)
    else:
        dec_f1_score.append((2*dec_tp[i])/((2*dec_tp[i])+dec_fp[i]+dec_fn[i]))
    i+=1

    
# The matrix is square, with all correct classifications along the upper-left to lower-right diagonal, in other words, predicted vs actual 
#therefore, we are selecting these values only


print("Evaluating Naive Bayes Model")

print("Precision:")
print(naive_precision[:1])
print("Recall:")
print(naive_recall[3:4])
print("F1 Score:")
print(naive_f1_score[3:4])

print("Evaluating Decision Tree Model")

print("Precision:")
print(dec_precision[2:3])
print("Recall:")
print(dec_recall[3:4])
print("F1 Score:")
print(dec_f1_score[:1])


Evaluating Naive Bayes Model
Precision:
[0.6666666666666666]
Recall:
[0.7777777777777778]
F1 Score:
[0.49122807017543857]
Evaluating Decision Tree Model
Precision:
[0.3684210526315789]
Recall:
[0.5555555555555556]
F1 Score:
[0.6857142857142857]


The best metric for this type of assignments is Recall as this is the fraction of the relevant information that are successfully retrieved (Wikipedia). The model that gives the higher Recall percentage is Naive Bayes, therefore, this model will be used for the second hotel 

_______________________________________________________________________________________________________________________
HOTEL 2: 

Amsterdam Court Hotel

226 West 50th Street, New York City, NY 10019-6702

Rating: 3.5
_______________________________________________________________________________________________________________________

In [17]:
dfe=pd.read_csv("Rivarola.Ester-A5_Hotel2.csv",usecols=[0,1],names=["rating","review"],encoding = "ISO-8859-1")
dfe.head(5)


,rating,review
0,4.0,Spent 10 days here over Christmas and New Year...
1,4.0,Clean and comfortable. Staff was friendly enou...
2,3.0,We needed a last minute hotel for a last minut...
3,1.0,I do not recommend staying here. This hotel ha...
4,5.0,"Terrific location, European sized room, basic,..."


In [18]:
english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer=super(TfidfVectorizer,self).build_analyzer()
        return lambda doc:(english_stemmer.stem(w) for w in analyzer(doc))
vectorizer=StemmedTfidfVectorizer(min_df=1,stop_words='english',decode_error='ignore')

In [19]:
#we needed to import numpy again as otherwise, the it was overwriting the data for dataset1

import numpy as np
target=[]
for i in dfe.rating:
    if(i==1):
        target.append("1")
    elif(i==2):
        target.append("2")
    elif(i==3):
        target.append("3")
    elif(i==4):
        target.append("4")
    else:
        target.append("5")
z=np.array(target)
a=vectorizer.fit_transform(dfe.review)

In [20]:
print(z.shape)
print(a.shape)

(200,)
(200, 1767)


In [21]:
#80% for training - 20% for testing

z_train,z_test,a_train,a_test=train_test_split(z,a,random_state=40)

naive_precision1=[]
naive_recall1=[]
naive_f1_score1=[]

dec_precision1=[]
dec_recall1=[]
dec_f1_score1=[]

# precision 
i=0
while(i<5):
    if(naive_tp[i]==0 and naive_fp[i]==0):
        naive_precision1.append(-1)
    else:
        naive_precision1.append(naive_tp[i]/(naive_fp[i]+naive_tp[i]))
    if(dec_tp[i]==0 and dec_fp[i]==0):
        dec_precision1.append(-1)
    else:
        dec_precision1.append(dec_tp[i]/(dec_fp[i]+dec_tp[i]))
        i+=1

    # recall
i=0
while(i<5):
    if(naive_tp[i]==0 and naive_fn[i]==0):
        naive_recall1.append(-1)
    else:
        naive_recall1.append(naive_tp[i]/(naive_fn[i]+naive_tp[i]))
    if(dec_tp[i]==0 and dec_fn[i]==0):
        dec_recall1.append(-1)
    else:
        dec_recall1.append(dec_tp[i]/(dec_fn[i]+dec_tp[i]))
        i+=1

    # f1 score
i=0
while(i<5):
    if(naive_tp[i]==0 and naive_fp[i]==0 and naive_fn[i]==0):
        naive_f1_score1.append(-1)
    else:
        naive_f1_score1.append((2*naive_tp[i])/((2*naive_tp[i])+naive_fp[i]+naive_fn[i]))
    if(dec_tp[i]==0 and dec_fp[i]==0 and dec_fn[i]==0):
        dec_f1_score1.append(-1)
    else:
        dec_f1_score1.append((2*dec_tp[i])/((2*dec_tp[i])+dec_fp[i]+dec_fn[i]))
    i+=1

print("Precision-Naive Bayes")
print(naive_precision1[:1])
print("Recall-Naive Bayes")
print(naive_recall1[2:3])
print("F1 Score-Naive Bayes")
print(naive_f1_score1[2:3])


Precision-Naive Bayes
[0.6666666666666666]
Recall-Naive Bayes
[0.5833333333333334]
F1 Score-Naive Bayes
[0.4307692307692308]


Recall values for the second hotel decreased but it still above 50% which makes it useful 